# Vaccine Temp pipeline objectives
Notes:

Report ID = [2137, 2111] (report numbers 38 and 39 respectively)

A:
1) export report: "Export facility info for temperature logs upload", #38 (2137)
2) Complete 3 transformations
    - add field: "redcap_event_name" = "facility_arm_1", for all records
    - add field: "facility_info_complete" = 2, for all records
    - set field "redcap_event_name" as second index
3) import into PID278

B:
1) export report: "Export storage info for temperature logs upload", #39 (2111)
2) Complete 4 transformations
    - add field: "redcap_event_name" = "month_year_arm_1", month spelled out, all lower case, 4 digit integer for year, global variable
    - add field: "storage_info_complete" = 2, for all records
    - rename columns according to:
        {
         "waiis_fac_name":"waiis_fac_name_s",
         "loc_vfc_pin":"loc_vfc_pin_s",
         "org_name":"org_name_s"
         }
     - set field "redcap_event_name" as second index
3) import into PID278


In [21]:
import redcap
from credentials import api_keys #dictionary of 4 api keys: prod_pid100, dev_pid100, prod_pid278, dev_pid278
from credentials import dev_url
from credentials import prod_url
from credentials import unique_event

In [11]:
prod100 = redcap.Project(prod_url, api_keys["prod_pid100"])
dev278 = redcap.Project(dev_url, api_keys["dev_pid278"])

# A) Report: Export Facility info for temperature logs upload, #38 (2137)

## Export Reports and Transform

In [17]:
def facility_info_transform(df_facility_info):
    """
    A single function to capture 3 transformations of facility_info event
    
    args:
        df_facility_info - dataframe of facility info report (report_id:2137)
    return:
        Dataframe - transformed dataframe
    """
    df_facility_info = df_facility_info.copy()
    
    # Create new event field, set all values to "facility_arm_1"
    df_facility_info["redcap_event_name"] = "facility_arm_1"
    # Create new field for complete/incomplete tag, and set all values to 2
    df_facility_info["facility_info_complete"] = 2
    # Move "redcap_event_name" to first column in prep for import
    col = df_facility_info.pop("redcap_event_name")
    df_facility_info.insert(0, col.name, col)
    
    return df_facility_info

In [18]:
# 1)Export Report #38 prod
df_facility_info = prod100.export_report(report_id=2137 ,format_type = "df")

In [19]:
# 2) complete several transformations
df_facility_info = facility_info_transform(df_facility_info)

In [20]:
# 3) import into dev environment
dev278.import_records(df_facility_info, import_format = "df")

{'count': 1737}

# B) Report: Export storage info for temperature logs upload: #39 (2111)

## Export Reports and Transform

In [12]:
def storage_info_transform(df_storage_info):
    """
    A single function to capture 5 transformations of storage_info event
    
    args:
        df_storage_info - dataframe of storage info report (report_id:2111)
    return:
        Dataframe - transformed dataframe
    """
    
    df_storage_info = df_storage_info.copy()
    
    # Create new event field, set all values to "facility_arm_1"
    df_storage_info["redcap_event_name"] = unique_event
    # Create new field for complete/incomplete tag, and set all values to 2
    df_storage_info["storage_info_complete"] = 2
    # Rename columns 
    mapper = {"waiis_fac_name":"waiis_fac_name_s", 
                 "loc_vfc_pin":"loc_vfc_pin_s",
                 "org_name":"org_name_s" }
    df_storage_info = df_storage_info.rename(columns = mapper)
    # move "redcap_event_name" to first column in prep for import
    col = df_storage_info.pop("redcap_event_name")
    df_storage_info.insert(0, col.name, col)
    # conver float fields to int
    float_fields = df_storage_info.dtypes[df_storage_info.dtypes == float].index
    df_storage_info.loc[:, float_fields] = df_storage_info.loc[:, float_fields].astype("Int64")
    
    return df_storage_info

In [13]:
# 1)Export Report #38 prod
df_storage_info = prod100.export_report(report_id = 2111 ,format_type = "df")

In [14]:
# 2) Complete transformation
df_storage_info = storage_info_transform(df_storage_info)

In [16]:
#3) import into dev environment
dev278.import_records(df_storage_info, import_format = "df")

{'count': 1737}